# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
from selenium import webdriver

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [18]:
def get_cleaned_up_data(dirty_string):
  return dirty_string.split(':')[-1].strip()

In [2]:
# The URL of the page contains the license number string, i.e. https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
driver = webdriver.Firefox()

# Get the page two times, because for some reason the first time we will 
# bounce back to the form
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display'
                      '.asp?mcrnumber=006179570C')
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display'
           '.asp?mcrnumber=006179570C')



In [19]:
# - The business name
business_name = driver.find_element_by_xpath('/html/body/div/div['
                                             '1]/div/table/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
# - Owner/operator
owner = driver.find_element_by_xpath('/html/body/div/div['
                                     '1]/div/table/tbody/tr/td/font/table['
                                     '2]/tbody/tr[3]/td[1]').text

# - Phone number
# Yes, this is not foolproof and fails later on, it is actually fixed in the 
# function version
phone_number = driver.find_element_by_xpath('/html/body/div/div['
                                            '1]/div/table/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
# - License status (Active, Expired, Etc)
licence_status = driver.find_element_by_xpath('/html/body/div/div['
                                              '1]/div/table/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
# - Physical address
address = driver.find_element_by_xpath('/html/body/div/div['
                                       '1]/div/table/tbody/tr/td/font/table['
                                       '3]/tbody/tr[2]/td[2]').text

# simple cleanup
business_name = get_cleaned_up_data(business_name)
owner = get_cleaned_up_data(owner)
phone_number = get_cleaned_up_data(phone_number)
licence_status = get_cleaned_up_data(licence_status)
address = get_cleaned_up_data(address)

In [20]:
print(business_name, owner, phone_number, licence_status, address)

B.D. SMITH TOWING BRANDT SMITH / OWNER 8173330706 Expired 13619 BRETT JACKSON RD.
FORT WORTH, TX. 76179


# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [22]:
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display'
           '.asp?mcrnumber=006179570C')
# - The business name
business_name = driver.find_element_by_xpath('/html/body/div/div['
                                             '1]/div/table/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
# - Owner/operator
owner = driver.find_element_by_xpath('/html/body/div/div['
                                     '1]/div/table/tbody/tr/td/font/table['
                                     '2]/tbody/tr[3]/td[1]').text

# - Phone number
phone_number = driver.find_element_by_xpath('/html/body/div/div['
                                            '1]/div/table/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
# - License status (Active, Expired, Etc)
licence_status = driver.find_element_by_xpath('/html/body/div/div['
                                              '1]/div/table/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
# - Physical address
address = driver.find_element_by_xpath('/html/body/div/div['
                                       '1]/div/table/tbody/tr/td/font/table['
                                       '3]/tbody/tr[2]/td[2]').text

# simple cleanup, use a dict, because we're gonna end up using a dict anyway,
#  I'd wager
extracted_data = {
  'business_name': get_cleaned_up_data(business_name),
  'owner': get_cleaned_up_data(owner),
  'phone': get_cleaned_up_data(phone_number),
  'licence_status': get_cleaned_up_data(licence_status),
  'address': get_cleaned_up_data(address)
}


print(extracted_data)

{'business_name': 'B.D. SMITH TOWING', 'owner': 'BRANDT SMITH / OWNER', 'phone': '8173330706', 'licence_status': 'Expired', 'address': '13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179'}


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [23]:
import pandas as pd

In [32]:
df_licence_numbers_subset = pd.read_csv('08-homework/trucks-subset.csv')

In [33]:
df_licence_numbers_subset

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [34]:
def create_url(licence_number):
  return 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber' \
         '={}'.format(licence_number)

In [35]:
df_licence_numbers_subset['url'] = df_licence_numbers_subset['TDLR Number'].apply(create_url)

In [36]:
df_licence_numbers_subset

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_d...
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_d...
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_d...


### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [31]:
# see above

## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [50]:
def scrape_licence_information(url):
  driver.get(url)
  # - The business name
  business_name = driver.find_element_by_xpath('/html/body/div/div['
                                               '1]/div/table/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
  # - Owner/operator
  owner = driver.find_element_by_xpath('/html/body/div/div['
                                       '1]/div/table/tbody/tr/td/font/table['
                                       '2]/tbody/tr[3]/td[1]').text

  # - Phone number
  phone_number = driver.find_element_by_xpath('/html/body/div/div['
                                              '1]/div/table/tbody/tr/td/font/table[2]/tbody/tr[last()]/td[1]').text
  # - License status (Active, Expired, Etc)
  licence_status = driver.find_element_by_xpath('/html/body/div/div['
                                                '1]/div/table/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
  # - Physical address
  address = driver.find_element_by_xpath('/html/body/div/div['
                                         '1]/div/table/tbody/tr/td/font/table['
                                         '3]/tbody/tr[2]/td[2]').text

  # simple cleanup, use a dict, because we're gonna end up using a dict anyway,
  #  I'd wager
  extracted_data = {
    'business_name': get_cleaned_up_data(business_name),
    'owner': get_cleaned_up_data(owner),
    'phone': get_cleaned_up_data(phone_number),
    'licence_status': get_cleaned_up_data(licence_status),
    'address': get_cleaned_up_data(address)
  }

  return pd.Series(extracted_data)


In [51]:
df_licence_numbers_subset['url'].apply(scrape_licence_information)

,business_name,owner,phone,licence_status,address
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST\nBONHAM, TX. 75418"
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179"
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD\nCANYON, TX. 79015"


## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [41]:
df_scraped_data = df_licence_numbers_subset['url'].apply(scrape_licence_information)

In [42]:
df_tow_trucks = df_licence_numbers_subset.join(df_scraped_data)

In [43]:
df_tow_trucks

,TDLR Number,url,business_name,owner,phone,licence_status,address
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_d...,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST\nBONHAM, TX. 75418"
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_d...,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD.\nFORT WORTH, TX. 76179"
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_d...,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD\nCANYON, TX. 79015"


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [44]:
df_tow_trucks.to_csv('08-homework/tow-trucks-extended.csv', index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [46]:
# yeah, yeah, it's fine

## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [47]:
df_all_numbers = pd.read_csv('08-homework/tow-trucks.csv')

In [54]:
df_all_numbers['TDLR Number']\
  .apply(create_url)\
  .apply(scrape_licence_information)\
  .join(df_all_numbers)\
  .to_csv('08-homework/tow-trucks-extended.csv', index=False)
